In [ ]:
from gba.client import MistralInstruct, Llama3Instruct, LlamaCppClient

# Proxy for 8-bit finetuned Mistral-7B-v0.1 planner
mistral_finetuned = MistralInstruct(
    llm=LlamaCppClient(url="http://localhost:8082/completion", temperature=-1),
)

# Proxy for 4-bit quantized Llama-3-8B-Instruct
llama3_instruct = Llama3Instruct(
    llm=LlamaCppClient(url="http://localhost:8084/completion", temperature=-1),
)

# Proxy for 8-bit quantized NexusRaven-V2-13B
nexus_raven = LlamaCppClient(url="http://localhost:8089/completion", temperature=-1)

In [ ]:
from gba.agent import Agent
from gba.client import ChatClient
from gba.planner import FineTunedPlanner
from gba.search import SearchEngine
from gba.store import DocumentStore
from gba.summary import ResultSummarizer
from gba.tools import *

from example_docs import DOCUMENTS
from example_funcs import create_event

store = DocumentStore(path=".chroma")
engine = SearchEngine(store=store)

if store.count() == 0:
    for i, document in enumerate(DOCUMENTS):
        store.add(identifier=str(i), document=document)

summarizer = ResultSummarizer(model=llama3_instruct)

tools = [
    AskTool(),
    CalculateTool(model=llama3_instruct, summarizer=None),
    FunctionCallTool(model=nexus_raven, fn=create_event),
    FunctionCallTool(model=nexus_raven, fn=engine.search_images),
    FunctionCallTool(model=nexus_raven, fn=engine.search_internet, summarizer=summarizer),
    RespondTool(model=llama3_instruct),
]

client = ChatClient(model=mistral_finetuned)
planner = FineTunedPlanner(client=client)
agent = Agent(planner=planner, tools=tools)

In [ ]:
agent.run("what is Leo DiCaprio's current girlfriend's age raised to the 0.24 power?")

In [ ]:
agent.run("How many years did sharks appear earlier on Earth, compared to octopuses?")

In [ ]:
agent.run("Add Martin's birthday party, starting Dec. 17th 2023 8pm, "
          "to my calendar and respond with a one-line invitation")

In [ ]:
agent.run("What are the 3 primary colors?")